In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
pd.set_option('display.max_columns', None)

In [2]:
os.getcwd()

'/Users/caseyh/Desktop/footyfootyfooty/data_wrangling'

In [3]:
seasons = sorted(os.listdir('/Users/caseyh/Desktop/footyfootyfooty/round_results/'))[-25:-1]

In [4]:
player_files = sorted(os.listdir('/Users/caseyh/Desktop/footyfootyfooty/player_stats/'))[-24:]

In [5]:
player_data = pd.DataFrame({})
for file in player_files:
    path = '/Users/caseyh/Desktop/footyfootyfooty/player_stats/'
    tmp = pd.read_csv(path + file)
    player_data = pd.concat([player_data, tmp])
player_data = player_data.reset_index(drop=True)

In [6]:
season_data = pd.DataFrame({})
for season in seasons:
    path = '/Users/caseyh/Desktop/footyfootyfooty/round_results/'
    tmp = pd.read_csv(path + season)
    season_data = pd.concat([season_data, tmp])
season_data = season_data.reset_index(drop=True)

season_data = season_data.dropna()

In [7]:
season_data['Date'] = season_data['Date'].str.extract(r'(\d{1,2} \w{3})')
season_data['Date'] = season_data['Date'] + ' ' + season_data['year'].apply(str)

In [8]:
def rearrange_date(x):
    return datetime.strptime(x, '%d %b %Y')

In [9]:
season_data['Date'] = season_data['Date'].apply(lambda x: rearrange_date(x))

In [10]:
teams = ['RCH', 'GEE', 'NTH', 'POR', 'GCS', 'MEL', 'GWS', 'HAW', 'STK',
       'CAR', 'BRL', 'COL', 'ADE', 'FRE', 'WBD', 'SYD', 'ESS', 'WCE']

In [11]:
# rearrange game table into both sides (double)
def split_row(data):
    output = pd.DataFrame({})
    for season in data['year'].unique():
        for r in data['round'].unique():
            r_season = data[(data['round'] == r)&(data['year']==season)]
            for i,j in [('Home','Away'), ('Away', 'Home')]:
                rounds = pd.DataFrame({})
                rounds['venue'] = r_season['Venue']
                rounds['team'] = r_season[f'{i}']
                rounds['opponent'] = r_season[f'{j}']
                rounds['pf'] = r_season[f'{i} Score']
                rounds['pa'] = r_season[f'{j} Score']
                rounds['margin'] = r_season[f'{i} Score'] - r_season[f'{j} Score']
                rounds['year'] = season
                rounds['round'] = r
                rounds['at_home'] = i
                rounds['date'] = r_season['Date']


                output = pd.concat([output, rounds])

    output = output.reset_index(drop = True)
    output['round'] = output['round'].str.replace('Round ','R')
    return output

In [12]:
output = split_row(season_data)

In [13]:
output['at_home'] = output['at_home'] == 'Home'

In [14]:
output['round'] = pd.Categorical(output['round'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'R9', 'R10', 'R11',
       'R12', 'R13', 'R14', 'R15', 'R16', 'R17', 'R18', 'R19', 'R20',
       'R21', 'R22', 'R23', 'R24','EF', 'QF', 'SF', 'PF', 'GF.0','GF'])
output = output.sort_values(['year','round'])

In [15]:
home_games = output[output['at_home']].reset_index(drop=True)
away_games = output[~output['at_home']].reset_index(drop=True)

In [16]:
home_games['game_id'] = home_games.at_home.rank(method="first", ascending=False).astype(int)
away_games['game_id'] = away_games.at_home.rank(method="first", ascending=False).astype(int)

In [17]:
games = pd.concat([home_games,away_games]).sort_values('game_id').reset_index(drop=True)

In [18]:
games['round'] = pd.Categorical(games['round'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'R9', 'R10', 'R11',
       'R12', 'R13', 'R14', 'R15', 'R16', 'R17', 'R18', 'R19', 'R20',
       'R21', 'R22', 'R23', 'R24','EF', 'QF', 'SF', 'PF', 'GF.0','GF'])
games = games.sort_values(['year','round'])

## Team Data
Aggregate player stats to get team stats

In [19]:
team_data = player_data.groupby(['year','round','team']).sum().reset_index()

In [20]:
team_data['round'] = pd.Categorical(team_data['round'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'R9', 'R10', 'R11',
       'R12', 'R13', 'R14', 'R15', 'R16', 'R17', 'R18', 'R19', 'R20',
       'R21', 'R22', 'R23', 'R24','EF', 'QF', 'SF', 'PF', 'GF.0','GF'])
team_data = team_data.sort_values(['year','round'])

In [21]:
games.loc[(games['pf']==games['pa'])&(games['round']=='GF'),'round'] = 'GF.0'

In [22]:
team_data.loc[(team_data['round']=='GF')&(team_data['year']==2010),'round']='GF.0'

In [23]:
games = games.merge(team_data, how = 'left', on=['year','round','team'])

In [24]:
# add rushed behind to behinds (only includes player behinds)
games['rushed behinds'] = games['pf'] - 6 * games['Goals'] - games['Behinds']
games['Behinds'] = games['Behinds'] + games['rushed behinds']

In [25]:
non_feature_cols = ['team', 'date', 'at_home', 'game_id', 'round', 'venue', 'opponent', 'year']
games = games.rename(columns={col: 'f_' + col for col in games if col not in non_feature_cols})

In [26]:
features = games[['date', 'game_id', 'team', 'opponent', 'venue', 'at_home']].copy()

In [27]:
match_results = games[['game_id', 'team', 'opponent', 'f_margin']]

## Exponential rolling avergae

In [28]:
# Define a function which returns a DataFrame with the expontential moving average for each numeric stat
def create_exp_weighted_avgs(df, span):
    # Create a copy of the df with only the game id and the team - we will add cols to this df
    ema_features = df[['game_id', 'team']].copy()

    feature_names = [col for col in df.columns if col.startswith('f_')] # Get a list of columns we will iterate over

    for feature_name in feature_names:
        feature_ema = (df.groupby('team')[feature_name]
                         .transform(lambda row: (row.ewm(span=span)
                                                    .mean()
                                                    .shift(1))))
        ema_features[feature_name] = feature_ema

    return ema_features

In [29]:
features_rolling_averages = create_exp_weighted_avgs(games, 10)

In [30]:
features = pd.merge(features, features_rolling_averages, on=['game_id', 'team'])

## Form between Teams

In [31]:
form_btwn_teams = games[['game_id', 'team', 'opponent', 'f_margin']].copy()

In [32]:
form_btwn_teams['f_form_margin_btwn_teams'] = (games.groupby(['team', 'opponent'])['f_margin'].transform(lambda row: row.rolling(5).mean().shift())
                                                .fillna(0))

In [33]:
form_btwn_teams['f_form_past_5_btwn_teams'] = \
(games.assign(win=lambda df: df.apply(lambda row: 1 if row.f_margin > 0 else 0, axis='columns'))
              .groupby(['team', 'opponent'])['win']
              .transform(lambda row: row.rolling(5).mean().shift() * 5)
              .fillna(0))

In [34]:
# Merge to our features df
features = pd.merge(features, form_btwn_teams.drop(columns=['f_margin']), on=['game_id', 'team', 'opponent'])

## Single Row

In [35]:
# Get each match on single rows
single_row_df = (games[['game_id', 'team', 'f_Inside 50s', 'f_Rebounds', 'f_Disposals', 'at_home', ]]
                    .query('at_home == 1')
                    .rename(columns={'team': 'home_team', 'f_Inside 50s': 'f_Inside 50s_home', 'f_Rebounds': 'f_Rebounds_home', 'f_Disposals': 'f_Disposals_home'})
                    .drop(columns='at_home')
                    .pipe(pd.merge, games[['game_id', 'team', 'f_Inside 50s', 'f_Rebounds', 'f_Disposals', 'at_home']]
                                    .query('at_home == 0')
                                    .rename(columns={'team': 'away_team', 'f_Inside 50s': 'f_Inside 50s_away', 'f_Rebounds': 'f_Rebounds_away', 'f_Disposals': 'f_Disposals_away'})
                                    .drop(columns='at_home'), on='game_id'))

In [36]:
single_row_df = single_row_df.assign(f_I50_efficiency_home=lambda df: df.f_Rebounds_away / df['f_Inside 50s_home'],
                                    f_I50_efficiency_away=lambda df: df.f_Rebounds_home / df['f_Inside 50s_away'])

feature_efficiency_cols = ['f_I50_efficiency_home', 'f_I50_efficiency_away']

In [37]:
# Now let's create an Expontentially Weighted Moving Average for these features - we will need to reshape our DataFrame to do this
efficiency_features_multi_row = (single_row_df[['game_id', 'home_team'] + feature_efficiency_cols]
                                    .rename(columns={
                                        'home_team': 'team',
                                        'f_I50_efficiency_home': 'f_I50_efficiency',
                                        'f_I50_efficiency_away': 'f_I50_efficiency_opponent',
                                    })
                                    .append((single_row_df[['game_id', 'away_team'] + feature_efficiency_cols]
                                                 .rename(columns={
                                                     'away_team': 'team',
                                                     'f_I50_efficiency_home': 'f_I50_efficiency_opponent',
                                                     'f_I50_efficiency_away': 'f_I50_efficiency',
                                                 })), sort=True)
                                    .sort_values(by='game_id')
                                    .reset_index(drop=True))

efficiency_features = efficiency_features_multi_row[['game_id', 'team']].copy()
feature_efficiency_cols = ['f_I50_efficiency', 'f_I50_efficiency_opponent']

for feature in feature_efficiency_cols:
    efficiency_features[feature] = (efficiency_features_multi_row.groupby('team')[feature]
                                        .transform(lambda row: row.ewm(span=10).mean().shift(1)))

/var/folders/ww/qljj89hs53b_mxcs8j00tyfw0000gn/T/ipykernel_97704/550343861.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  efficiency_features_multi_row = (single_row_df[['game_id', 'home_team'] + feature_efficiency_cols]


In [38]:
# Get feature efficiency df back onto single rows
efficiency_features = pd.merge(efficiency_features, games[['game_id', 'team', 'at_home']], on=['game_id', 'team'])
efficiency_features_single_row = (efficiency_features.query('at_home == 1')
                                    .rename(columns={
                                        'team': 'home_team', 
                                        'f_I50_efficiency': 'f_I50_efficiency_home',
                                        'f_I50_efficiency_opponent': 'f_R50_efficiency_home'})
                                    .drop(columns='at_home')
                                    .pipe(pd.merge, (efficiency_features.query('at_home == 0')
                                                        .rename(columns={
                                                            'team': 'away_team',
                                                            'f_I50_efficiency': 'f_I50_efficiency_away',
                                                            'f_I50_efficiency_opponent': 'f_R50_efficiency_away'})
                                                        .drop(columns='at_home')), on='game_id'))

## Elo Rating

In [39]:
# Define a function which finds the elo for each team in each game and returns a dictionary with the game ID as a key and the
# elos as the key's value, in a list. It also outputs the probabilities and a dictionary of the final elos for each team
def elo_applier(df, k_factor):
    # Initialise a dictionary with default elos for each team
    elo_dict = {team: 1500 for team in df['team'].unique()}
    elos, elo_probs = {}, {}

    # Get a home and away dataframe so that we can get the teams on the same row
    home_df = df.loc[df.at_home == 1, ['team', 'game_id', 'f_margin', 'at_home']].rename(columns={'team': 'home_team'})
    away_df = df.loc[df.at_home == 0, ['team', 'game_id']].rename(columns={'team': 'away_team'})

    df = (pd.merge(home_df, away_df, on='game_id')
            .sort_values(by='game_id')
            .drop_duplicates(subset='game_id', keep='first')
            .reset_index(drop=True))

    # Loop over the rows in the DataFrame
    for index, row in df.iterrows():
        # Get the Game ID
        game_id = row['game_id']

        # Get the margin
        margin = row['f_margin']

        # If the game already has the elos for the home and away team in the elos dictionary, go to the next game
        if game_id in elos.keys():
            continue

        # Get the team and opposition
        home_team = row['home_team']
        away_team = row['away_team']

        # Get the team and opposition elo score
        home_team_elo = elo_dict[home_team]
        away_team_elo = elo_dict[away_team]

        # Calculated the probability of winning for the team and opposition
        prob_win_home = 1 / (1 + 10**((away_team_elo - home_team_elo) / 400))
        prob_win_away = 1 - prob_win_home

        # Add the elos and probabilities our elos dictionary and elo_probs dictionary based on the Game ID
        elos[game_id] = [home_team_elo, away_team_elo]
        elo_probs[game_id] = [prob_win_home, prob_win_away]

        # Calculate the new elos of each team
        if margin > 0: # Home team wins; update both teams' elo
            new_home_team_elo = home_team_elo + k_factor*(1 - prob_win_home)
            new_away_team_elo = away_team_elo + k_factor*(0 - prob_win_away)
        elif margin < 0: # Away team wins; update both teams' elo
            new_home_team_elo = home_team_elo + k_factor*(0 - prob_win_home)
            new_away_team_elo = away_team_elo + k_factor*(1 - prob_win_away)
        elif margin == 0: # Drawn game' update both teams' elo
            new_home_team_elo = home_team_elo + k_factor*(0.5 - prob_win_home)
            new_away_team_elo = away_team_elo + k_factor*(0.5 - prob_win_away)
        else: # no result yet
            new_home_team_elo = home_team_elo
            new_away_team_elo = away_team_elo
        # Update elos in elo dictionary
        elo_dict[home_team] = new_home_team_elo
        elo_dict[away_team] = new_away_team_elo

    return elos, elo_probs, elo_dict

In [40]:
# Use the elo applier function to get the elos and elo probabilities for each game - we will map these later
elos, probs, elo_dict = elo_applier(games, 30)

In [41]:
one_line_cols = ['game_id', 'team', 'at_home'] + [col for col in features if col.startswith('f_')]

# Get all features onto individual rows for each match
features_one_line = (features.loc[features.at_home == 1, one_line_cols]
                     .rename(columns={'team': 'home_team'})
                     .drop(columns='at_home')
                     .pipe(pd.merge, (features.loc[features.at_home == 0, one_line_cols]
                                              .drop(columns='at_home')
                                              .rename(columns={'team': 'away_team'})
                                              .rename(columns={col: col+'_away' for col in features.columns if col.startswith('f_')})), on='game_id')
                    .drop(columns=['f_form_margin_btwn_teams_away', 'f_form_past_5_btwn_teams_away']))

# Add our created features - elo, efficiency etc.
features_one_line = (features_one_line.assign(f_elo_home=lambda df: df.game_id.map(elos).apply(lambda x: x[0]),
                                            f_elo_away=lambda df: df.game_id.map(elos).apply(lambda x: x[1]))
                                      .pipe(pd.merge, efficiency_features_single_row, on=['game_id', 'home_team', 'away_team'])
                                      .pipe(pd.merge, games.loc[games.at_home == 1, ['game_id', 'date', 'round', 'venue']], on=['game_id'])
#                                       .dropna()
                                      .reset_index(drop=True)
                                      .assign(season=lambda df: df.date.apply(lambda row: row.year)))

ordered_cols = [col for col in features_one_line if col[:2] != 'f_'] + [col for col in features_one_line if col.startswith('f_')]

feature_df = features_one_line[ordered_cols]

In [42]:
feature_df = feature_df.loc[~(feature_df['f_Disposals'].isna())]

In [43]:
# Create differential df - this df is the home features - the away features
diff_cols = [col for col in feature_df.columns if col + '_away' in feature_df.columns and col.startswith('f_')]
non_diff_cols = [col for col in feature_df.columns if col not in diff_cols and col[:-5] not in diff_cols]

diff_df = feature_df[non_diff_cols].copy()

for col in diff_cols:
    diff_df[col+'_diff'] = feature_df[col] - feature_df[col+'_away']

# Prep Model

In [45]:
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
# from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_selection import RFECV
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn import feature_selection
from sklearn import metrics
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
import warnings
warnings.filterwarnings('ignore')

In [46]:
match_results = (home_games[['game_id','pf','pa','margin']]
     .assign(result=lambda df: df.apply(lambda row: 1 if row['pf'] > row['pa'] else 0, axis=1)))

In [47]:
# Merge result column to feature_df
feature_df = pd.merge(diff_df, match_results, on='game_id')

In [48]:
# Create our test and train sets from our afl DataFrame; drop the columns which leak the result, duplicates, and the advanced
# stats which don't have data until 2015

feature_columns = [col for col in feature_df if col.startswith('f_')]

# Create our test set
test_x = feature_df.loc[feature_df.game_id >= 4626, ['game_id'] + feature_columns]
test_y = feature_df.loc[feature_df.game_id >= 4626, 'margin']

# Create our train set
X = feature_df.loc[(feature_df.game_id < 4626), ['game_id'] + feature_columns]
y = feature_df.loc[(feature_df.game_id < 4626), 'margin']

# Scale features
scaler = StandardScaler()
X[feature_columns] = scaler.fit_transform(X[feature_columns])
test_x[feature_columns] = scaler.transform(test_x[feature_columns])

In [155]:
best_algos = find_best_algorithms(classifiers, X, y)
best_algos

,Mean Log Loss,Log Loss Std,Algorithm
0,0.595562,2.168812e-02,LogisticRegressionCV
1,0.597428,2.290373e-02,LinearDiscriminantAnalysis
2,0.610307,1.907438e-02,ExtraTreesClassifier
3,0.621466,2.431478e-02,RandomForestClassifier
4,0.681384,1.017178e-03,NuSVC
5,0.685889,6.441717e-03,AdaBoostClassifier
6,0.693147,8.245947e-10,GaussianProcessClassifier
7,0.711937,1.378235e-01,GradientBoostingClassifier
8,0.954778,7.063676e-02,BaggingClassifier
9,1.265322,7.857978e-02,BernoulliNB


In [53]:
# Define a function which optimises the hyperparameters of our chosen algorithms
def optimise_hyperparameters(train_x, train_y, algorithms, parameters):
    kfold = StratifiedKFold(n_splits=5)
    best_estimators = []

    for alg, params in zip(algorithms, parameters):
        gs = GridSearchCV(alg, param_grid=params, cv=kfold, scoring='neg_log_loss', verbose=1)
        gs.fit(train_x, train_y)
        best_estimators.append(gs.best_estimator_)
    return best_estimators

rf_grid = {
    'bootstrap': [True],
    'max_depth': [100],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100]
}
# Add our algorithms and parameters to lists to be used in our function
alg_list = [RandomForestRegressor()]
param_list = [rf_grid]

In [ ]:
# Find the best estimators, then add our other estimators which don't need optimisation
best_estimators = optimise_hyperparameters(X, y, alg_list, param_list)

In [57]:
rf_best_params = {
    'bootstrap': True,
    'max_depth': 100,
    'min_samples_leaf': 4,
    'min_samples_split': 10,
    'n_estimators': 100
}

In [ ]:
rf_best_params = best_estimators[0].get_params()
rf_best_params

In [58]:
rf = RandomForestRegressor(**rf_best_params)
rf.fit(X, y)
final_predictions = rf.predict(test_x)

# Prediction Season Results

In [59]:
future = home_games.loc[home_games.game_id >= 4626]

In [60]:
future['prediction'] = final_predictions

In [62]:
future

,venue,team,opponent,pf,pa,margin,year,round,at_home,date,game_id,prediction
4625,MCG,COL,CAR,NaN,NaN,NaN,2023,R20,True,2023-07-28,4626,12.071073
4626,Mars Stadium,WBD,GWS,NaN,NaN,NaN,2023,R20,True,2023-07-29,4627,10.398906
4627,GMHBA Stadium,GEE,FRE,NaN,NaN,NaN,2023,R20,True,2023-07-29,4628,43.009363
4628,Heritage Bank Stadium,GCS,BRL,NaN,NaN,NaN,2023,R20,True,2023-07-29,4629,-9.992064
4629,Adelaide Oval,ADE,POR,NaN,NaN,NaN,2023,R20,True,2023-07-29,4630,-4.144000
4630,Marvel Stadium,ESS,SYD,NaN,NaN,NaN,2023,R20,True,2023-07-29,4631,-23.838663
4631,Marvel Stadium,HAW,STK,NaN,NaN,NaN,2023,R20,True,2023-07-30,4632,5.738299
4632,Optus Stadium,WCE,NTH,NaN,NaN,NaN,2023,R20,True,2023-07-30,4633,-15.255202
4633,MCG,RCH,MEL,NaN,NaN,NaN,2023,R20,True,2023-07-30,4634,2.090366
4634,Marvel Stadium,WBD,RCH,NaN,NaN,NaN,2023,R21,True,2023-08-04,4635,6.540847


# Get Future Ladder

In [697]:
afl_games = pd.concat([home_games,away_games]).sort_values('game_id').reset_index(drop=True)

In [698]:
outcomes = afl_games.merge(future[['prediction','game_id','at_home']], on=['game_id','at_home'],how='left')

In [648]:
for game in range(4626, outcomes.shape[0]+1):
    outcomes.loc[(~outcomes['at_home'])&(outcomes['game_id']==game), 'prediction'] = \
 abs(outcomes[(outcomes['at_home'])&(outcomes['game_id']==game)]['prediction'].values - 1)

In [649]:
outcomes.loc[outcomes['prediction'].isna(), 'prediction'] = (np.select(
    [outcomes[outcomes['prediction'].isna()]['margin']>0, 
     outcomes[outcomes['prediction'].isna()]['margin']<0],[1,0],0.5))

In [583]:
ladder = outcomes[['team','round','year']]

In [584]:
ladder = ladder[~ladder['round'].isin(['EF','QF','SF','PF','GF','GF.0'])].reset_index(drop=True)

In [585]:
ladder = (ladder.join(outcomes[~outcomes['round']
                                 .isin(['EF','QF','SF','PF','GF','GF.0'])]
                                 .reset_index(drop=True)
                                 .groupby(['year','team'])[['pf','pa','prediction']].cumsum()))

In [586]:
ladder['per'] = ladder['pf']/ladder['pa']

In [587]:
ladder = ladder.sort_values(['year','round','prediction','per'],ascending=[True,True,False,False])

In [588]:
ladder['position'] = ladder.groupby(['year','round']).prediction.rank(method="first", ascending=False)

## Predict Grand Final

# Add data and check winners 

### Add first week of finals

In [460]:
dic = {'year':2023, 'team_pos':[1,2,5,6],'opponent_pos':[4,3,8,7],
            'round':['QF','QF','EF','EF']}

In [461]:
final_fixtures_23 = pd.DataFrame(dic,columns=['venue', 'team_pos', 'opponent_pos',
                                              'margin','year','round'])

In [462]:
last_round = ladder[(ladder['year']==2023)&(ladder['round']=='R24')][['team','position']]

In [463]:
final_fixtures_23 = (final_fixtures_23.merge(last_round, left_on='team_pos',right_on='position',how='left').drop(columns='position')
                  .merge(last_round.rename(columns={'team':'opponent'}), left_on='opponent_pos',right_on='position',how='left') 
                  .drop(columns=['position','team_pos','opponent_pos'])
                  .rename(columns={'team':'Home','opponent':'Away'}))

In [464]:
final_fixtures_23

,venue,margin,year,round,Home,Away
0,NaN,NaN,2023,QF,COL,MEL
1,NaN,NaN,2023,QF,POR,BRL
2,NaN,NaN,2023,EF,WBD,RCH
3,NaN,NaN,2023,EF,CAR,GEE


In [481]:
season_data = pd.concat([season_data,final_fixtures_23])

### Add Second week of finals

In [577]:
future.tail()

,venue,team,opponent,pf,pa,margin,year,round,at_home,date,game_id,prediction
4669,GMHBA Stadium,GEE,WBD,NaN,NaN,NaN,2023,R24,True,2023-08-25,4670,1
4670,NaN,WBD,RCH,NaN,NaN,NaN,2023,EF,True,NaT,4671,1
4671,NaN,CAR,GEE,NaN,NaN,NaN,2023,EF,True,NaT,4672,0
4672,NaN,COL,MEL,NaN,NaN,NaN,2023,QF,True,NaT,4673,1
4673,NaN,POR,BRL,NaN,NaN,NaN,2023,QF,True,NaT,4674,0


In [601]:
dic = {'year':2023, 'Home':['MEL','POR'],'Away':['WBD','GEE'],
            'round':['SF','SF']}

In [602]:
final_fixtures_23 = pd.DataFrame(dic,columns=['venue', 'Home', 'Away',
                                              'margin','year','round'])

In [603]:
season_data = pd.concat([season_data,final_fixtures_23])

### Add Third week of finals

In [745]:
future.tail(10)

,venue,team,opponent,pf,pa,margin,year,round,at_home,date,game_id,prediction
4669,GMHBA Stadium,GEE,WBD,NaN,NaN,NaN,2023,R24,True,2023-08-25,4670,1
4670,NaN,WBD,RCH,NaN,NaN,NaN,2023,EF,True,NaT,4671,1
4671,NaN,CAR,GEE,NaN,NaN,NaN,2023,EF,True,NaT,4672,0
4672,NaN,COL,MEL,NaN,NaN,NaN,2023,QF,True,NaT,4673,1
4673,NaN,POR,BRL,NaN,NaN,NaN,2023,QF,True,NaT,4674,0
4674,NaN,MEL,WBD,NaN,NaN,NaN,2023,SF,True,NaT,4675,1
4675,NaN,POR,GEE,NaN,NaN,NaN,2023,SF,True,NaT,4676,0
4676,NaN,BRL,MEL,NaN,NaN,NaN,2023,PF,True,NaT,4677,1
4677,NaN,COL,POR,NaN,NaN,NaN,2023,PF,True,NaT,4678,1
4678,NaN,COL,BRL,NaN,NaN,NaN,2023,GF,True,NaT,4679,0


In [746]:
dic = {'year':2023, 'Home':['BRL','COL'],'Away':['MEL','GEE'],
            'round':['PF','PF']}

In [747]:
final_fixtures_23 = pd.DataFrame(dic,columns=['venue', 'Home', 'Away',
                                              'margin','year','round'])

In [ ]:
season_data.merge(final_fixtures_23)

In [653]:
season_data = pd.concat([season_data,final_fixtures_23])

### Add LAST week of finals

In [699]:
future.tail()

,venue,team,opponent,pf,pa,margin,year,round,at_home,date,game_id,prediction
4673,NaN,POR,BRL,NaN,NaN,NaN,2023,QF,True,NaT,4674,1
4674,NaN,MEL,WBD,NaN,NaN,NaN,2023,SF,True,NaT,4675,1
4675,NaN,POR,GEE,NaN,NaN,NaN,2023,SF,True,NaT,4676,1
4676,NaN,BRL,MEL,NaN,NaN,NaN,2023,PF,True,NaT,4677,1
4677,NaN,COL,POR,NaN,NaN,NaN,2023,PF,True,NaT,4678,1


In [700]:
dic = {'year':2023, 'Home':['COL'],'Away':['BRL'],
            'round':['GF']}

In [701]:
final_fixtures_23 = pd.DataFrame(dic,columns=['venue', 'Home', 'Away',
                                              'margin','year','round'])

In [702]:
season_data = pd.concat([season_data,final_fixtures_23])

In [703]:
season_data

,Date,Venue,Crowd,round,year,Home,Away,Home Score,Away Score,venue,margin
0,2000-03-08,MCG,41648.0,Round 1,2000,MEL,RCH,92.0,94.0,NaN,NaN
1,2000-03-09,Marvel Stadium,43012.0,Round 1,2000,ESS,POR,156.0,62.0,NaN,NaN
2,2000-03-10,MCG,20580.0,Round 1,2000,NTH,WCE,111.0,154.0,NaN,NaN
3,2000-03-11,Domain Stadium,19777.0,Round 1,2000,FRE,GEE,107.0,129.0,NaN,NaN
4,2000-03-11,AAMI Stadium,37222.0,Round 1,2000,ADE,WBD,108.0,131.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
0,NaT,NaN,NaN,SF,2023,MEL,WBD,NaN,NaN,NaN,NaN
1,NaT,NaN,NaN,SF,2023,POR,GEE,NaN,NaN,NaN,NaN
0,NaT,NaN,NaN,PF,2023,BRL,MEL,NaN,NaN,NaN,NaN
1,NaT,NaN,NaN,PF,2023,COL,POR,NaN,NaN,NaN,NaN
